In [31]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from xmitgcm import utils
from xmitgcm import open_mdsdataset
from xgcm import Grid
from xhistogram.xarray import histogram
from dask.diagnostics import ProgressBar

In [34]:
stats = xr.open_dataset("data/tracer_stats.nc")
ids = xr.open_dataset("data/upcanyon_section.nc")

#### Compute distance along path

In [35]:
R = 6371000.
def r(x, y, x0, y0):
    dλ = x - x0
    dϕ = y - y0
    ϕ = (y + y0)/2.
    dx = 2.*np.pi*R * dλ / (360.) * np.cos(ϕ)
    dy = 2.*np.pi*R * dϕ / (360.)
    return 2.*( (x>x0) - 0.5) * np.sqrt(dx**2 + dy**2)

In [37]:
rids = ids.sel(XC=stats['XC_bar'][0].values, method="nearest")
ids = ids.assign_coords({'r': r(ids['XC'], ids['YC'], rids['XC'], rids['YC'])})

In [38]:
ids = ids.swap_dims({'XC':'r'})

In [52]:
import scipy.io as sio
import astropy
from astropy.time import Time

In [124]:
fctd = sio.loadmat("data/FCTD_transects.mat", simplify_cells=True)['transects']

def get_trajectory(tjname):
    try:
        lon = fctd[tjname]['fctd_lon']
        lat = fctd[tjname]['fctd_lat']
        time = fctd[tjname]['time']
    except:
        lon = fctd[tjname]['fish_lon']
        lat = fctd[tjname]['fish_lat']
        time = fctd[tjname]['time']

    t0 = Time(0.0000, format="decimalyear", scale='utc').plot_date - 1.; # shift by one day to convert to matlab datenum format
    time = (Time(t0 + time , format='plot_date', scale='utc').datetime).astype("datetime64")
    
    trajectory = xr.Dataset({
        'XC': xr.DataArray(np.append(lon + 1.e-8*np.random.rand(lon.size), np.nan), dims='cast'),
        'YC': xr.DataArray(np.append(lat + 1.e-8*np.random.rand(lon.size), np.nan), dims='cast'),
        'time': xr.DataArray(np.append(time, time[-1] + np.timedelta64('15', 'm')), dims='cast')
    })
    return trajectory

In [125]:
traj = xr.concat(
    [get_trajectory(k) for k in list(fctd.keys())[:-1]],
    dim='cast'
)

/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "utctai" yielded 1 of "dubious year (Note 3)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "taiutc" yielded 1 of "dubious year (Note 4)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA function "dtf2d" yielded 1 of "dubious year (Note 6)"
  warnings.warn('ERFA function "{}" yielded {}'.format(func_name, wmsg),
/home/hdrake/.conda/envs/rockall/lib/python3.10/site-packages/erfa/core.py:154: ErfaWarning: ERFA 

In [146]:
rtraj = r(traj['XC'], rids['YC'], rids['XC'], rids['YC']).assign_coords({'time':traj['time']}).swap_dims({'cast':'time'})

In [171]:
fig = plt.figure(figsize=(10, 5))

for i, h in enumerate(np.arange(0, 42, 0.2)):
    Δt = np.int64(np.rint(h*3600.))
    pc = (ids['TRAC01'].interp(time=time_release + Δt).where(ids['hFacC']!=0.)).plot(
        cmap="Greens", norm=matplotlib.colors.LogNorm(vmax=1e-2, vmin=1e-4)
    )

    (ids['THETA'].where(ids['hFacC']>=0.5).interp(time=time_release + Δt)).plot.contour(
        levels=np.arange(2., 6., 0.1), colors='k', alpha=0.5, linewidths=0.75,
    )
    (ids['THETA'].where(ids['hFacC']>=0.5).interp(time=time_release + Δt)).plot.contour(
        levels=[3.76], colors='k', linewidths=1.5
    )


    plt.axvline(rtraj.interp(time=time_release + Δt), color="C3", alpha=0.75)

    plt.ylim(-2000, -1500)
    plt.ylabel("elevation [m]")
    plt.xlabel("along-canyon distance [m]")
    hstr = str(round(h,1))
    plt.title(f"Tracer at {hstr} hours", loc="left")
    plt.title("")
        
    plt.savefig(f"figures/up-canyon-tracer/{str(i).zfill(4)}.png", dpi=200, bbox_inches="tight")
    fig.clf()

<Figure size 720x360 with 0 Axes>